# Чанкирование и векторизация
### Данный этап выполняется в Google Colab с T4. Следует также добавить путь к документам из базы знаний в DRIVE_PATH и FOLDER_NAME.

In [2]:
# Установка необходимых пакетов (в Colab)
!pip install -q langchain faiss-cpu sentence-transformers langchain-community

# Монтируем Google Drive для доступа к файлам
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Примечание: В каждом текстовом файле базы знаний в первой строке писалось название документа, после чего делался отступ в 3 пустые строки.

**Чанкирование и векторизация:**

In [ ]:
# Указываем путь к нужной папке в Google Drive
DRIVE_PATH = '/content/drive/MyDrive'
FOLDER_NAME = 'FOLDER_NAME'
DOCS_FOLDER = f"{DRIVE_PATH}/{FOLDER_NAME}"

print(f"Проверка файлов в папке: {DOCS_FOLDER}")
!ls "{DOCS_FOLDER}"  # Проверяем содержимое папки

# Функция для извлечения названия документа
def extract_document_title(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    title = ""
    empty_line_count = 0
    for line in lines:
        if line.strip() == "":
            empty_line_count += 1
            if empty_line_count == 3:
                break
        else:
            title += line.strip() + " "
            empty_line_count = 0
    return title.strip()

# Функция для чанкирования текста
def chunk_text(file_path, chunk_size=900, chunk_overlap=250):
    # Извлекаем название документа
    doc_title = extract_document_title(file_path)
    # Читаем весь текст файла
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    # Удаляем название и все до трех пустых строк
    text = text.split('\n\n\n', 1)[1] if '\n\n\n' in text else text
    # Настраиваем сплиттер
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    # Разбиваем текст на чанки
    chunks = splitter.split_text(text)
    # Формируем список чанков с метаданными
    chunk_documents = []
    for i, chunk in enumerate(chunks):
        chunk_documents.append({
            "text": chunk,
            "metadata": {
                "document": doc_title,
                "chunk_number": i + 1
            }
        })
    return chunk_documents

# Загрузка всех чанков из папки
def load_all_chunks(docs_folder):
    import os
    all_chunks = []
    for filename in os.listdir(docs_folder):
        if filename.endswith('.txt'):
            file_path = os.path.join(docs_folder, filename)
            print(f"Обработка: {filename}")
            chunks = chunk_text(file_path)
            all_chunks.extend(chunks)
    return all_chunks

# Инициализация модели эмбеддингов
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={'device': 'cuda'}  # Используем GPU Colab
)



# Создание векторной базы
import faiss
import numpy as np
from tqdm.auto import tqdm
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.schema import Document
from langchain_community.vectorstores import FAISS

# Загружаем все чанки
all_chunks = load_all_chunks(DOCS_FOLDER)

# Извлекаем текст и метаданные
texts = [chunk['text'] for chunk in all_chunks]
metadatas = [chunk['metadata'] for chunk in all_chunks]

# Создаем объекты Document для LangChain
documents = []
for i, (text, metadata) in enumerate(zip(texts, metadatas)):
    documents.append(Document(
        page_content=text,
        metadata=metadata
    ))

# Генерируем эмбеддинги
print("Генерация эмбеддингов...")
vectors = embedding_model.embed_documents(texts)

# Нормализуем для косинусного сходства
vectors = np.array(vectors)
vectors = vectors / np.linalg.norm(vectors, axis=1, keepdims=True)

# Создаем FAISS индекс
dimension = vectors.shape[1]
index = faiss.IndexFlatL2(dimension)

# Добавляем векторы пакетами
batch_size = 50
for i in tqdm(range(0, len(vectors), batch_size), desc="Создание индекса"):
    batch = vectors[i:i + batch_size]
    index.add(batch)

# Создаем ID для каждого документа
ids = [str(i) for i in range(len(documents))]
# Создаем InMemoryDocstore
docstore = InMemoryDocstore(dict(zip(ids, documents)))
# Создаем индекс для соответствия ID
index_to_id = {i: str(i) for i in range(len(documents))}

# инициализация FAISS
db = FAISS(
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_id,
    embedding_function=embedding_model.embed_query
)

# Сохраняем в Google Drive
SAVE_PATH = f"{DRIVE_PATH}/faiss_index_cosine"
db.save_local(SAVE_PATH)
print(f"\nИндекс сохранен в: {SAVE_PATH}")

**Реализация функции поиска:**

In [ ]:
# Функция поиска
def search_with_cosine_similarity(query, db, top_k=3, cosine_threshold=0.7):
    # Получаем вектор запроса
    query_embedding = db.embedding_function(query)
    # Нормализуем вектор запроса
    query_embedding = np.array(query_embedding) / np.linalg.norm(query_embedding)
    # Выполняем поиск с использованием similarity_search_with_score для получения L2-дистанции
    results = db.similarity_search_with_score(query, k=top_k)
    print("\nНайденные релевантные чанки:")
    for i, (doc, l2_distance) in enumerate(results, 1):
        cosine_sim = 1 - (l2_distance ** 2) / 2
        print(f"\nЧанк {i} (сходство: {cosine_sim:.4f}):")
        print(f"Источник: {doc.metadata['document']}")
        print(f"Текст: {doc.page_content[:200]}...")

    if results and (1 - (results[0][1]**2)/2) > cosine_threshold:
        return [res[0] for res in results]
    return None

**Тестирование поиска:**

In [ ]:
# Пример поиска
query = "Кто может быть залогодателем"
print(f"\nПоиск по запросу: '{query}'")
context = search_with_cosine_similarity(query, db)

if context:
    print("\nКонтекст для ответа:")
    for i, doc in enumerate(context, 1):
        print(f"{i}. {doc.page_content[:150]}...")
else:
    print("Релевантные документы не найдены")